In [3]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

label_map = {
    'equation1': 'x^2',
    'equation2': '\sqrt(x)',
    'equation3': '\sqrt[3]{x}',
    'equation4': '\\ frac{x}{y}',
    'equation5': '\\ frac{1}{2}',
    'equation6': '\ ax+b=0',
    'equation7': '\ ax^2+bx+c=0',
    'equation8': '\delta=b^2-4ac',
    'equation9': '\(ab)^n=a^nb^n',
    'equation10': '(a^m)^n=a^mn',
}

# Load and preprocess the dataset with padding
def load_data(data_dir):
    images = []
    labels = []
    
    for label in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label)
        if os.path.isdir(label_path):
            for img_file in os.listdir(label_path):
                img_path = os.path.join(label_path, img_file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                h, w = img.shape
                desired_h, desired_w = 128, 128
                
                # If the image is smaller than the desired size, calculate padding
                pad_h = max(0, (desired_h - h) // 2)
                pad_w = max(0, (desired_w - w) // 2)

                # Resize image to fit within desired dimensions, keeping aspect ratio
                img_resized = cv2.resize(img, (desired_w - 2 * pad_w, desired_h - 2 * pad_h))
                
                # Create a blank canvas with padding
                canvas = np.zeros((desired_h, desired_w), dtype=np.uint8)
                canvas[pad_h:pad_h + img_resized.shape[0], pad_w:pad_w + img_resized.shape[1]] = img_resized

                # Normalize
                canvas = canvas / 255.0
                images.append(canvas)
                labels.append(label_map[label])

    images = np.expand_dims(np.array(images), axis=-1)  # Add channel dimension
    return images, np.array(labels)


# Define the main function
def main():
    data_dir = r'C:\Users\Harshitha D\OneDrive\Desktop\Kakracholi\dataset\data_prepared'  # Path to your dataset folder
    X, y = load_data(data_dir)

    # Encode the labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    # Build the CNN model
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(128, 128, 1)),  # Updated input shape
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(label_map), activation='softmax')  # Output layer for number of classes
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train, epochs=55, batch_size=16, validation_data=(X_test, y_test))

    # Save the model
    model.save('handwritten_equation_model.h5')
    print("Model saved as 'handwritten_equation_model.h5'")

# Run the main function
if __name__ == '__main__':
    main()


Epoch 1/55
2/2 [==============================] - 1s 309ms/step - loss: 2.2906 - accuracy: 0.1250 - val_loss: 2.0864 - val_accuracy: 0.2857
Epoch 2/55
2/2 [==============================] - 0s 143ms/step - loss: 1.6674 - accuracy: 0.6667 - val_loss: 1.5685 - val_accuracy: 0.4286
Epoch 3/55
2/2 [==============================] - 0s 147ms/step - loss: 0.8042 - accuracy: 0.7917 - val_loss: 0.7893 - val_accuracy: 0.7143
Epoch 4/55
2/2 [==============================] - 0s 147ms/step - loss: 0.2788 - accuracy: 0.9167 - val_loss: 1.5240 - val_accuracy: 0.2857
Epoch 5/55
2/2 [==============================] - 0s 147ms/step - loss: 0.0805 - accuracy: 1.0000 - val_loss: 4.0898 - val_accuracy: 0.4286
Epoch 6/55
2/2 [==============================] - 0s 138ms/step - loss: 0.6461 - accuracy: 0.8333 - val_loss: 2.9458 - val_accuracy: 0.5714
Epoch 7/55
2/2 [==============================] - 0s 158ms/step - loss: 0.0229 - accuracy: 1.0000 - val_loss: 2.4780 - val_accuracy: 0.7143
Epoch 8/55
2/2 [====